In [1]:
# делаем import необходимых библиотек
import os
import mlflow
import psycopg
import pandas as pd
from dotenv import load_dotenv
from sklearn.metrics import roc_auc_score, f1_score, log_loss, recall_score, precision_score, confusion_matrix

In [2]:
# подгружаем .env
load_dotenv()

connection = {"sslmode": "require", "target_session_attrs": "read-write"}

postgres_credentials = {
    "host": os.environ.get('DB_DESTINATION_HOST'), 
    "port": os.environ.get('DB_DESTINATION_PORT'),
    "dbname": os.environ.get('DB_DESTINATION_NAME'),
    "user": os.environ.get('DB_DESTINATION_USER'),
    "password": os.environ.get('DB_DESTINATION_PASSWORD'),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

In [65]:
# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

In [16]:
# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]


In [17]:
# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns) 

In [18]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,28173,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0
1,28174,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
2,28175,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,No,No,No,No,Male,0,No,No,No,1
3,28176,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,...,Yes,Yes,No,No,Male,0,No,No,None,0
4,28177,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,...,No,No,No,No,Female,0,No,No,No,1


In [3]:
# scripts/fit.py

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from catboost import CatBoostClassifier
import yaml
import os
import joblib

In [4]:
# загрузка модели
with open('/home/mle-user/mle_projects/mle-dvc/models/fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd)

In [5]:
# загружаем результат предыдущего шага: inital_data.csv
data = pd.read_csv('/home/mle-user/mle_projects/mle-dvc/data/initial_data.csv')

In [6]:
data.head()

,id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,7020,2020-01-01,NaN,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,...,No,No,No,No,Female,0,Yes,No,No,0
1,7021,2017-04-01,NaN,One year,No,Mailed check,56.95,1889.50,DSL,Yes,...,Yes,No,No,No,Male,0,No,No,No,0
2,7022,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,...,No,No,No,No,Male,0,No,No,No,1
3,7023,2016-05-01,NaN,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,...,Yes,Yes,No,No,Male,0,No,No,No,0
4,7024,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,...,No,No,No,No,Female,0,No,No,No,1


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    data,
    data['target'],
    stratify=data['target']
)


In [10]:
pip install scikit-learn==1.4.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.1
    Uninstalling scikit-learn-1.3.1:
      Successfully uninstalled scikit-learn-1.3.1
Note: you may need to restart the kernel to use updated packages.


In [9]:
prediction = model.predict(X_test)
proba = model.predict_proba(X_test)

In [10]:
_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()

In [11]:
err2

0.2655270655270655

In [12]:
proba[:,0]

array([0.99980479, 0.9998092 , 0.99982364, ..., 0.99985206, 0.99976065,
       0.99985515])

In [13]:
# импортируйте необходимые вам модули

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel() # ваш код здесь #
auc = roc_auc_score(y_test, proba[:,0]) # ваш код здесь #
precision = precision_score(y_test, prediction) # ваш код здесь #
recall = recall_score(y_test, prediction) # ваш код здесь #
f1 = f1_score(y_test, prediction) # ваш код здесь #
logloss = log_loss(y_test, prediction) # ваш код здесь #

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

In [14]:
metrics

{'err1': 0.0,
 'err2': 0.2655270655270655,
 'auc': 0.0,
 'precision': 1.0,
 'recall': 1.0,
 'f1': 1.0,
 'logloss': 2.2204460492503136e-16}

In [15]:
# задаём название эксперимента и имя запуска для логирования в MLflow

EXPERIMENT_NAME = "churn_kruglikovAlex" # ваш код здесь (напишите своё уникальное имя эксперимента)
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_kruglikovAlex"

In [ ]:
cat_features = X_train.select_dtypes(include='object')
potential_binary_features = X_train.nunique() == 2

In [22]:
binary_cat_features = data[potential_binary_features[potential_binary_features].index]
other_cat_features = cat_features
num_features = data.select_dtypes(['float','int'])

In [31]:
binary_cat_features.select_dtypes(['float','int']).columns

Index(['senior_citizen', 'target'], dtype='object')

In [32]:
# уберем из числовых бинарные признаки
num_features.drop(columns=binary_cat_features.select_dtypes(['float','int']).columns, inplace=True) 

In [36]:
preprocessor = ColumnTransformer(
    [
        ('binary', OneHotEncoder(drop='if_binary'), binary_cat_features.columns.tolist()),
        ('cat', CatBoostEncoder(return_df=False), other_cat_features.columns.tolist()),
        ('num', StandardScaler(), num_features.columns.tolist())
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

In [41]:
X_tr_transformed = preprocessor.fit_transform(X_train, y_train)

In [44]:
X_tr = pd.DataFrame(X_tr_transformed, columns=preprocessor.get_feature_names_out())

In [46]:
# ваш код здесь

pip_requirements = "../requirements.txt" # ваш код здесь
signature = mlflow.models.infer_signature(
    X_tr,
    prediction.astype(int)
)
# ваш код здесь
input_example = X_test[:10] # ваш код здесь
metadata = metadata = {'model_type': 'monthly'} # ваш код здесь


In [47]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [59]:
# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
#experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) # ваш код здесь
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

In [60]:
experiment_id

'1'

In [51]:
class CatboostModelProba(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        super().__init__()
        self._model = model

    def predict(self, context, model_input):
        import numpy as np
        predictions = np.sqrt(self._model.predict(model_input))

        return predictions

In [52]:
custom_model = CatboostModelProba(model) 

In [61]:
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id # ваш код здесь
    
    model_info = mlflow.pyfunc.log_model( 
        # ваш код здесь #
        python_model=custom_model,
        #cb_model=model,
        signature=signature,
        pip_requirements=pip_requirements,
        metadata = metadata,
        input_example = input_example,
        artifact_path="models",
        await_registration_for=60,
        registered_model_name=REGISTRY_MODEL_NAME,
        )

    # логируем метрики эксперимента
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(metrics)

    # логируем файл как артефакт эксперимента — 'users_churn.csv'
    mlflow.log_artifact("/home/mle-user/mle_projects/mle-dvc/data/initial_data.csv", "dataframe")


Registered model 'churn_model_kruglikovAlex' already exists. Creating a new version of this model...
2025/07/08 21:20:34 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_kruglikovAlex, version 2
Created version '2' of model 'churn_model_kruglikovAlex'.


In [62]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) # ваш код здесь

In [63]:
# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert run.info.status == "FINISHED" # ваш код здесь

In [64]:
run.info.status

'FINISHED'

In [65]:
run_id

'74fd405fdfa34a609caf709078a04aed'

In [68]:
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

2025/07/08 21:22:04 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.4.0, required: scikit-learn==1.3.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [73]:
X_tr.dtypes

paperless_billing_Yes           float64
internet_service_Fiber optic    float64
online_security_Yes             float64
online_backup_Yes               float64
device_protection_Yes           float64
tech_support_Yes                float64
streaming_tv_Yes                float64
streaming_movies_Yes            float64
gender_Male                     float64
senior_citizen_1                float64
partner_Yes                     float64
dependents_Yes                  float64
multiple_lines_Yes              float64
target_1                        float64
begin_date                      float64
end_date                        float64
type                            float64
paperless_billing               float64
payment_method                  float64
internet_service                float64
online_security                 float64
online_backup                   float64
device_protection               float64
tech_support                    float64
streaming_tv                    float64


In [72]:
model_predictions = loaded_model.predict(X_tr)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [100]:
model_predictions.astype(int)

array([0, 0, 0, ..., 1, 0, 0])

In [101]:
model_predictions.dtype

dtype('float64')

In [102]:
assert model_predictions.astype(int).dtype == int

print(model_predictions[:10])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
